<a href="https://colab.research.google.com/github/EvinduArunoda/pump-it-up-170473p/blob/main/moracse_training_170473p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

path = '/content/drive/MyDrive/Machine Learning Project/Data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
#from sklearn.ensemble import VotingClassifier
#from sklearn.neighbors import KNeighborsClassifier

test = pd.read_csv(path+"/test_set_values.csv")
train_x = pd.read_csv(path+"/training_set_values.csv")
train_y = pd.read_csv(path+"/training_set_labels.csv")

train = train_y.merge(train_x, on='id')
#train = pd.read_csv(path+"/clean.csv")


In [3]:
#target.head()

In [4]:
target = train.pop('status_group')

train['train']=1
test['train']=0

combined = pd.concat([train, test])

target.shape

(59400,)

In [5]:
target.head()

0        functional
1        functional
2        functional
3    non functional
4        functional
Name: status_group, dtype: object

Pre-processing

In [6]:
combined['gps_height'].replace(0.0, np.nan, inplace=True)
combined['population'].replace(0.0, np.nan, inplace=True)
combined['amount_tsh'].replace(0.0, np.nan, inplace=True)

combined["gps_height"].fillna(combined.groupby(['region', 'district_code'])["gps_height"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined.groupby(['region'])["gps_height"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined["gps_height"].mean(), inplace=True)
combined["population"].fillna(combined.groupby(['region', 'district_code'])["population"].transform("median"), inplace=True)
combined["population"].fillna(combined.groupby(['region'])["population"].transform("median"), inplace=True)
combined["population"].fillna(combined["population"].median(), inplace=True)
combined["amount_tsh"].fillna(combined.groupby(['region', 'district_code'])["amount_tsh"].transform("median"), inplace=True)
combined["amount_tsh"].fillna(combined.groupby(['region'])["amount_tsh"].transform("median"), inplace=True)
combined["amount_tsh"].fillna(combined["amount_tsh"].median(), inplace=True)

features=['amount_tsh', 'gps_height', 'population']
scaler = MinMaxScaler(feature_range=(0,20))
combined[features] = scaler.fit_transform(combined[features])

combined['longitude'].replace(0.0, np.nan, inplace=True)
combined['latitude'].replace(0.0, np.nan, inplace=True)
combined['construction_year'].replace(0.0, np.nan, inplace=True)

combined["latitude"].fillna(combined.groupby(['region', 'district_code'])["latitude"].transform("mean"), inplace=True)
combined["longitude"].fillna(combined.groupby(['region', 'district_code'])["longitude"].transform("mean"), inplace=True)
combined["longitude"].fillna(combined.groupby(['region'])["longitude"].transform("mean"), inplace=True)
combined["construction_year"].fillna(combined.groupby(['region', 'district_code'])["construction_year"].transform("median"), inplace=True)
combined["construction_year"].fillna(combined.groupby(['region'])["construction_year"].transform("median"), inplace=True)
combined["construction_year"].fillna(combined.groupby(['district_code'])["construction_year"].transform("median"), inplace=True)
combined["construction_year"].fillna(combined["construction_year"].median(), inplace=True)

combined['date_recorded'] = pd.to_datetime(combined['date_recorded'])
combined['operational_year'] = combined.date_recorded.dt.year - combined.construction_year

#DATA Cleaning
waste_features=['wpt_name','num_private','subvillage','region_code','recorded_by']
#since region and region_code are representing the same thing
combined.drop(waste_features,axis=1,inplace=True)
combined.drop('management_group',axis=1,inplace=True)
combined.drop('extraction_type_group',axis=1,inplace=True)
combined.drop('extraction_type_class',axis=1,inplace=True)
combined.drop('payment',axis=1,inplace=True)
combined.drop('quality_group',axis=1,inplace=True)
combined.drop('quantity_group',axis=1,inplace=True)
combined.drop('source_type',axis=1,inplace=True)
combined.drop('source_class',axis=1,inplace=True)
combined.drop('waterpoint_type_group',axis=1,inplace=True)
combined.drop('ward',axis=1,inplace=True)
combined.drop('installer',axis=1,inplace=True)
combined.drop('public_meeting', axis=1, inplace=True)
combined.drop('permit', axis=1, inplace=True)

#For every categorical response, convert the levels to lower case, in case there is random capitalization
combined.waterpoint_type = combined.waterpoint_type.str.lower()
combined.funder = combined.funder.str.lower()
combined.basin = combined.basin.str.lower()
combined.region = combined.region.str.lower()
combined.source = combined.source.str.lower()
combined.lga = combined.lga.str.lower()
combined.management = combined.management.str.lower()
combined.quantity = combined.quantity.str.lower()
combined.water_quality = combined.water_quality.str.lower()
combined.payment_type=combined.payment_type.str.lower()
combined.extraction_type=combined.extraction_type.str.lower()

combined["funder"].fillna("other", inplace=True)
combined["scheme_management"].fillna("other", inplace=True)
print(combined.isnull().sum())
combined.drop('date_recorded', axis=1, inplace=True)
combined.drop('construction_year', axis=1, inplace=True)
combined.info()
combined.quantity.head(5)

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
longitude                0
latitude                 0
basin                    0
region                   0
district_code            0
lga                      0
population               0
scheme_management        0
scheme_name          35258
construction_year        0
extraction_type          0
management               0
payment_type             0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
train                    0
operational_year         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 74250 non-null  int64  
 1   amount_tsh         74250 non-null  float64
 2   funder             74250 non-null

0          enough
1    insufficient
2          enough
3             dry
4        seasonal
Name: quantity, dtype: object

In [7]:
combined.shape

(74250, 22)

In [8]:
combined.head()

,id,amount_tsh,funder,gps_height,longitude,latitude,basin,region,district_code,lga,population,scheme_management,scheme_name,extraction_type,management,payment_type,water_quality,quantity,source,waterpoint_type,train,operational_year
0,69572,0.342846,roman,10.324381,34.938093,-9.856322,lake nyasa,iringa,5,ludewa,0.070822,VWC,Roman,gravity,vwc,annually,soft,enough,spring,communal standpipe,1,12.0
1,8776,0.011417,grumeti,10.387164,34.698766,-2.147466,lake victoria,mara,2,serengeti,0.182957,Other,NaN,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,1,3.0
2,34310,0.001417,lottery club,5.413324,37.460664,-3.821329,pangani,manyara,4,simanjiro,0.163284,VWC,Nyumba ya mungu pipe scheme,gravity,vwc,per bucket,soft,enough,dam,communal standpipe multiple,1,4.0
3,67743,0.002846,unicef,2.462504,38.486161,-11.155298,ruvuma / southern coast,mtwara,63,nanyumbu,0.037378,VWC,NaN,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,1,27.0
4,19728,0.014274,action in a,7.995705,31.130847,-1.825359,lake victoria,kagera,1,karagwe,0.130496,other,NaN,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,1,8.0


In [9]:
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
combined.district_code.head(5)

0    0
1    1
2    2
3    3
4    4
Name: district_code, dtype: int64

In [10]:
##### Fix ####
combined.drop(["scheme_name"], axis=1, inplace=True)

train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
#define X train
X = train_df
#y_train indentified at the beginning 
y = target

In [12]:
model_rfc = RandomForestClassifier(n_estimators=1000)

In [13]:
model = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', 
                      num_class = 4, maximize = False, eval_metric = 'merror', eta = .2,
                      max_depth = 14, colsample_bytree = .4)

In [14]:
#knn=KNeighborsClassifier(n_neighbors=1000)

In [15]:
#eclf1 = VotingClassifier(estimators=[('xg', model), ('rf', model_rfc)],weights=[1,2], voting='hard')

In [16]:
cross_val_score(model_rfc, X, y, cv=3)

array([0.80419192, 0.80383838, 0.79984848])

In [ ]:
X.head()
model_rfc.fit(X,y)
X.info()
importances = model_rfc.feature_importances_
importances

In [ ]:
importances = model_rfc.feature_importances_
importances
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')

In [ ]:
#for xgboost

print(cross_val_score(model, X, y, cv=3))
model.fit(X,y)
importances = model.feature_importances_
importances
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')

In [ ]:
X_test=test_df

In [ ]:
model_rfc.fit(X,y)

In [ ]:
a=X_test['id']
X_test.drop(['id'],axis=1, inplace=True)
y_pred = model.predict(X_test)

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_pred['id']=a
y_pred.columns=['status_group','id']
y_pred=y_pred[['id','status_group']]

In [ ]:
y_pred.info()

In [ ]:
pd.DataFrame(y_pred).to_csv(path+"/submission_new.csv",index=False)